In [1]:
import numpy as np
import imutils
import cv2
import argparse

def stitch(images, ratio_dist=0.75, threshold = 4.0, draw_kp = False, type_s="H"): #main function which calls the others and stitch
    (imageB, imageA) = images
    (kpsA, featuresA) = detectAndDescribe(imageA)
    (kpsB, featuresB) = detectAndDescribe(imageB)

    M = matchKeypoints(kpsA, kpsB,featuresA, featuresB, ratio_dist, threshold, (imageB.shape,imageA.shape))

    if M is None:
        return (None,None)
    else:
        (matches, H, status, dist_) = M
        
        if type_s=="V":
            if imageB.shape[1]>=imageA.shape[1]:
                result = cv2.warpPerspective(imageA, H,(imageB.shape[1], imageA.shape[0] + imageB.shape[0]-dist_))
            else:
                result = cv2.warpPerspective(imageA, H,(imageA.shape[1], imageA.shape[0] + imageB.shape[0]-dist_))
        else: 
            if imageA.shape[0]>=imageB.shape[0]:
                result = cv2.warpPerspective(imageA, H,(imageA.shape[1] + imageB.shape[1]-dist_, imageA.shape[0]))
            else:
                result = cv2.warpPerspective(imageA, H,(imageA.shape[1] + imageB.shape[1], imageB.shape[0]))
        
        result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
        

        if draw_kp:
            vis = drawMatches(imageA, imageB, kpsA, kpsB, matches,status)
            return (result, vis)

        return (result, None)
    
def detectAndDescribe(image): #find feature points in each image

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    descriptor = cv2.xfeatures2d.SIFT_create()
    (kps, features) = descriptor.detectAndCompute(image, None)

    return (kps, features)
    
def matchKeypoints(kpsA, kpsB, featuresA, featuresB,ratio, threshold,shapes):#match feature points between the two images and estimate the homography matrices

    matcher = cv2.DescriptorMatcher_create("BruteForce")
    rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
    shapeB,_=shapes
    matches = []
    
    for m in rawMatches: 
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            matches.append((m[0].trainIdx, m[0].queryIdx))
    
    if len(matches) >= 4:
        
        kpsA_list=[kpsA[i] for (_, i) in matches]
        kpsB_list=[kpsB[i] for (i, _) in matches]
        ptsA=np.array([kp.pt for kp in kpsA_list],dtype=np.float32)
        ptsB=np.array([kp.pt for kp in kpsB_list],dtype=np.float32)
        
        if type_s=="V":
            dist_=int(shapeB[0]-np.max(ptsB[0,:])-np.max(ptsA[0,:]))
        else:
            dist_=int((shapeB[1]-np.min(ptsB[:,0]))-(np.min(ptsA[:,0])))#to 'crop' the black zone in the stitched image
        
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, threshold)
        

        return (matches, H, status, dist_)
    else:
        return None
    
def drawMatches(imageA, imageB, kpsA, kpsB, matches, status): #draw the matched feature points in the images
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    for ((trainIdx, queryIdx), s) in zip(matches, status):

        if s == 1:
            
            ptA = (int(kpsA[queryIdx].pt[0]), int(kpsA[queryIdx].pt[1]))
            ptB = (int(kpsB[trainIdx].pt[0]) + wA, int(kpsB[trainIdx].pt[1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)
    
    return vis
    
#load images to stitch
imageA = cv2.imread('C:\\Users\\Administrateur\\Documents\\MASTER\\color_images\\cat1.jpg')
imageB = cv2.imread('C:\\Users\\Administrateur\\Documents\\MASTER\\color_images\\cat2.jpg')
type_s=input("By default images are stitch horizontally, press 'V' to stitch vertically or press any key to skip")

# stitch the images together to create a panorama

(result,vis) = stitch([imageA, imageB], 0.75, 4.0, True,type_s)
if result is not None:

    cv2.imshow("Image A", imageA)
    cv2.waitKey(0)
    cv2.imshow("Image B", imageB)
    cv2.waitKey(0)
    cv2.imshow("Keypoint Matches", vis)
    cv2.imshow("Result", result)
    cv2.waitKey(0)
    cv2.imwrite("Panorama_H.jpg",result)
else:
    print("No features points")

By default images are stitch horizontally, press 'V' to stitch vertically or press any key to skipV
